<a href="https://colab.research.google.com/github/dmarx/notebooks/blob/animate_diff/AnimateDiff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone -b dev https://github.com/camenduru/animatediff
!pip install einops omegaconf safetensors diffusers[torch]==0.11.1 transformers
!apt -y install -qq aria2
!rm -rf /content/animatediff/models/StableDiffusion
!git clone -b fp16 https://huggingface.co/runwayml/stable-diffusion-v1-5 /content/animatediff/models/StableDiffusion/
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/mm_sd_v14.ckpt -d /content/animatediff/models/Motion_Module -o mm_sd_v14.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/mm_sd_v15.ckpt -d /content/animatediff/models/Motion_Module -o mm_sd_v15.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/toonyou_beta3.safetensors -d /content/animatediff/models/DreamBooth_LoRA -o toonyou_beta3.safetensors


In [ ]:
!cd /content/animatediff; python -m scripts.animate --config /content/animatediff/configs/prompts/1-ToonYou.yaml --pretrained_model_path /content/animatediff/models/StableDiffusion --L 16 --W 256 --H 256

In [ ]:
!cd /content/animatediff; python -m scripts.animate --help

In [ ]:
#!pip install napm
import napm
napm.pseudoinstall_git_repo("https://github.com/guoyww/animatediff/", package_name='animatediff', add_install_dir_to_path=True)

In [ ]:
#import animatediff.scripts.animate
import napm
import argparse
import datetime
import inspect
import os
from omegaconf import OmegaConf

import torch

import diffusers
from diffusers import AutoencoderKL, DDIMScheduler

from tqdm.auto import tqdm
from transformers import CLIPTextModel, CLIPTokenizer

from animatediff.models.unet import UNet3DConditionModel
from animatediff.pipelines.pipeline_animation import AnimationPipeline
from animatediff.utils.util import save_videos_grid
from animatediff.utils.convert_from_ckpt import convert_ldm_unet_checkpoint, convert_ldm_clip_checkpoint, convert_ldm_vae_checkpoint
from animatediff.utils.convert_lora_safetensor_to_diffusers import convert_lora

from einops import rearrange, repeat

import csv, pdb, glob
from safetensors import safe_open
import math
from pathlib import Path

In [ ]:
from types import SimpleNamespace

# --pretrained_model_path /content/animatediff/models/StableDiffusion --L 16 --W 256 --H 256
args = SimpleNamespace()
args.pretrained_model_path = "/content/animatediff_github/models/StableDiffusion"
args.L = 16
args.W = 256
args.H = 256

#args.noise_scheduler_kwargs = OmegaConf.to_container(OmegaConf.load("/content/animatediff/configs/noise
args

In [ ]:
  # base: ""
  # path: "models/DreamBooth_LoRA/toonyou_beta3.safetensors"
  # motion_module:
  #   - "models/Motion_Module/mm_sd_v14.ckpt"
  #   - "models/Motion_Module/mm_sd_v15.ckpt"

  # seed:           [10788741199826055526, 6520604954829636163, 6519455744612555650, 16372571278361863751]
  # steps:          25
  # guidance_scale: 7.5

  # prompt:
  #   - "best quality, masterpiece, 1girl, looking at viewer, blurry background, upper body, contemporary, dress"
  #   - "masterpiece, best quality, 1girl, solo, cherry blossoms, hanami, pink flower, white flower, spring season, wisteria, petals, flower, plum blossoms, outdoors, falling petals, white hair, black eyes,"
  #   - "best quality, masterpiece, 1boy, formal, abstract, looking at viewer, masculine, marble pattern"
  #   - "best quality, masterpiece, 1girl, cloudy sky, dandelion, contrapposto, alternate hairstyle,"

  # n_prompt:
  #   - ""
  #   - "badhandv4,easynegative,ng_deepnegative_v1_75t,verybadimagenegative_v1.3, bad-artist, bad_prompt_version2-neg, teeth"
  #   - ""
  #   - ""

In [ ]:
import napm
dir(napm)
cfg = napm.config.NapmConfig().load()
PKG_ROOT = Path(cfg['packages']['animatediff']['install_dir'])

In [ ]:
model_config = SimpleNamespace()
model_config.path = ''
model_config.motion_module = "/content/animatediff_github/models/Motion_Module/mm_sd_v15.ckpt"  #PKG_ROOT/"models/Motion_Module/mm_sd_v15.ckpt"
model_config.seed = 10788741199826055526
model_config.steps = 25
model_config.guidance_scale = 7.5
model_config.prompt = "best quality, masterpiece, 1girl, looking at viewer, blurry background, upper body, contemporary, dress"
model_config.n_prompt = ""
#unet_additional_kwargs = {}

#parser.add_argument("--inference_config",      type=str, default="configs/inference/inference.yaml")
#inference_config = SimpleNamespace()
inference_config = OmegaConf.load(PKG_ROOT/"configs/inference/inference.yaml")
inference_config

In [ ]:
tokenizer    = CLIPTokenizer.from_pretrained(args.pretrained_model_path, subfolder="tokenizer")
text_encoder = CLIPTextModel.from_pretrained(args.pretrained_model_path, subfolder="text_encoder")
vae          = AutoencoderKL.from_pretrained(args.pretrained_model_path, subfolder="vae")
unet         = UNet3DConditionModel.from_pretrained_2d(args.pretrained_model_path, subfolder="unet", unet_additional_kwargs=OmegaConf.to_container(inference_config.get("unet_additional_kwargs", {})))

pipeline = AnimationPipeline(
  vae=vae, text_encoder=text_encoder, tokenizer=tokenizer, unet=unet,
  scheduler=DDIMScheduler(**OmegaConf.to_container(inference_config.noise_scheduler_kwargs)),
).to("cuda")

In [ ]:
func_args = SimpleNamespace
motion_module_state_dict = torch.load(model_config.motion_module, map_location="cpu")
if "global_step" in motion_module_state_dict:
  func_args.update({"global_step": motion_module_state_dict["global_step"]})
missing, unexpected = pipeline.unet.load_state_dict(motion_module_state_dict, strict=False)
assert len(unexpected) == 0

In [ ]:
 # 1.2 T2I
if model_config.path != "":
    if model_config.path.endswith(".ckpt"):
        state_dict = torch.load(model_config.path)
        pipeline.unet.load_state_dict(state_dict)

    elif model_config.path.endswith(".safetensors"):
        state_dict = {}
        with safe_open(model_config.path, framework="pt", device="cpu") as f:
            for key in f.keys():
                state_dict[key] = f.get_tensor(key)

        is_lora = all("lora" in k for k in state_dict.keys())
        if not is_lora:
            base_state_dict = state_dict
        else:
            base_state_dict = {}
            with safe_open(model_config.base, framework="pt", device="cpu") as f:
                for key in f.keys():
                    base_state_dict[key] = f.get_tensor(key)

        # vae
        converted_vae_checkpoint = convert_ldm_vae_checkpoint(base_state_dict, pipeline.vae.config)
        pipeline.vae.load_state_dict(converted_vae_checkpoint)
        # unet
        converted_unet_checkpoint = convert_ldm_unet_checkpoint(base_state_dict, pipeline.unet.config)
        pipeline.unet.load_state_dict(converted_unet_checkpoint, strict=False)
        # text_model
        pipeline.text_encoder = convert_ldm_clip_checkpoint(base_state_dict)

        # import pdb
        # pdb.set_trace()
        if is_lora:
            pipeline = convert_lora(pipeline, state_dict, alpha=model_config.lora_alpha)

pipeline.to("cuda")
### <<< create validation pipeline <<< ###

In [ ]:
sample = pipeline(
    prompt=model_config.prompt,
    negative_prompt     = model_config.n_prompt,
    num_inference_steps = model_config.steps,
    guidance_scale      = model_config.guidance_scale,
    width               = args.W,
    height              = args.H,
    video_length        = args.L,
).videos


In [ ]:
samples = torch.concat([sample])
savedir="."
save_videos_grid(samples, f"{savedir}/sample.gif", n_rows=1)